In [1]:
import pandas as pd
import fastai

df = pd.read_csv('..\\data\\election_results\\1976-2020-senate.csv', 
                 skiprows=lambda x: 1<=x<=3048, # to start the year 2000 because party politics weird before then
                 # omit data after 2012 because don't have CF scores after that 
                 #skipfooter= 768, # last 2012 row is row2861, there are 3629 rows so need to skip the bottom 768 rows
                 encoding='latin1')

In [2]:
df

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party_detailed,writein,mode,candidatevotes,totalvotes,unofficial,version,party_simplified
0,2016,ALABAMA,AL,1,63,41,US SENATE,statewide,gen,False,RON CRUMPTON,DEMOCRAT,False,total,748709,2087444,False,20210114,DEMOCRAT
1,2016,ALABAMA,AL,1,63,41,US SENATE,statewide,gen,False,RICHARD C. SHELBY,REPUBLICAN,False,total,1335104,2087444,False,20210114,REPUBLICAN
2,2016,ALABAMA,AL,1,63,41,US SENATE,statewide,gen,False,NaN,NaN,True,total,3631,2087444,False,20210114,OTHER
3,2016,ALASKA,AK,2,94,81,US SENATE,statewide,gen,False,JOE MILLER,LIBERTARIAN,False,total,90825,311441,False,20210114,LIBERTARIAN
4,2016,ALASKA,AK,2,94,81,US SENATE,statewide,gen,False,TED GIANOUTSOS,INDEPENDENT,False,total,1758,311441,False,20210114,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,2020,WYOMING,WY,56,83,68,US SENATE,statewide,gen,False,UNDER VOTES,NaN,False,total,6401,278503,False,20210114,OTHER
577,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,True,KELLY LOEFFLER,REPUBLICAN,False,total,2194848,4483294,True,20210114,REPUBLICAN
578,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,True,RAPHAEL WARNOCK,DEMOCRAT,False,total,2288446,4483294,True,20210114,DEMOCRAT
579,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,False,DAVID A. PERDUE,REPUBLICAN,False,total,2213979,4483241,True,20210114,REPUBLICAN


In [2]:
df = df[(df.special==False) & (df.writein==False) & (df.unofficial==False)]
df
df = df.loc[:,['year','state_po','candidate','candidatevotes','party_simplified','state']]
df
df = df[(df.party_simplified=='REPUBLICAN') | (df.party_simplified=='DEMOCRAT')]
df.head(10)

to_del = []
i = 0
while i<=df.shape[0]-3: # gives indexOB error when I try to go to the end. Works for now because I know (manually) that last one that needs to be deleted is within this range
    start = i # temp variable so we can get length of block later
    count = 0
    curr_state = df.iloc[i].state_po
    #from IPython.core.debugger import Pdb; Pdb().set_trace()
    while df.iloc[i].state_po==curr_state:
        count+=1
        i+=1
    if count == 2:
        pass#i+=1 # move onto next state
    else:
        print(start)
        to_del.extend(range(start,i)) # utilize non-inclusive upper bound because at this point, 
                                      # the next state has already been moved onto by incrementing i 
                                      # in the inner while loop above
        #i+=1 # AGH this is wrong cause if length less than 2, already moved onto next state
to_del
#df

# because .drop() uses an iterable of index LABELS, must do:
to_drop = [df.index[i] for i in to_del]
to_drop

elections = df.drop(to_drop)
elections

32
136
149
166


,year,state_po,candidate,candidatevotes,party_simplified,state
0,2016,AL,RON CRUMPTON,748709,DEMOCRAT,ALABAMA
1,2016,AL,RICHARD C. SHELBY,1335104,REPUBLICAN,ALABAMA
5,2016,AK,RAY METCALFE,36200,DEMOCRAT,ALASKA
8,2016,AK,LISA MURKOWSKI,138149,REPUBLICAN,ALASKA
11,2016,AZ,ANN KIRKPATRICK,1031245,DEMOCRAT,ARIZONA
...,...,...,...,...,...,...
562,2020,TX,"MARY ""MJ"" HEGAR",4888764,DEMOCRAT,TEXAS
566,2020,VA,MARK R. WARNER,2466500,DEMOCRAT,VIRGINIA
567,2020,VA,DANIEL M. GADE,1934199,REPUBLICAN,VIRGINIA
569,2020,WV,SHELLEY MOORE CAPITO,547454,REPUBLICAN,WEST VIRGINIA


In [3]:
elections.insert(3,'Dem_candidate',[None for x in range(elections.shape[0])])
elections.insert(4,'Repub_candidate',[None for x in range(elections.shape[0])])
elections.insert(5,'Dem_votes',[None for x in range(elections.shape[0])])
elections.insert(6,'Repub_votes',[None for x in range(elections.shape[0])])

In [4]:
i = 1 # because going to look at previous one
while i<=elections.shape[0]:
    if elections.iloc[i,8]=='DEMOCRAT': # party_simplified
        elections.iloc[i,3]=elections.iloc[i,2] # Dem_candidate, candidate
        elections.iloc[i,4]=elections.iloc[i-1,2] # Repub_candidate, candidate
        elections.iloc[i,5]=elections.iloc[i,7] # Dem_votes, candidatevotes
        elections.iloc[i,6]=elections.iloc[i-1,7] # Repub_votes, candidatevotes
    elif elections.iloc[i,8]=='REPUBLICAN':
        elections.iloc[i,3]=elections.iloc[i-1,2] # Dem_candidate, candidate
        elections.iloc[i,4]=elections.iloc[i,2] # Repub_candidate, candidate
        elections.iloc[i,5]=elections.iloc[i-1,7] # Dem_votes, candidatevotes
        elections.iloc[i,6]=elections.iloc[i,7] # Repub_votes, candidatevotes
    i+=2

In [5]:
elections = elections.iloc[1::2]

In [6]:
elections.insert(8,'Repub%of_two-party_vote', elections.Repub_votes/(elections.Dem_votes+elections.Repub_votes))
elections = elections.drop(['candidate','Dem_votes','Repub_votes','candidatevotes','party_simplified'], axis=1) # dropping in-place now is causing errors later

In [7]:
elections

,year,state_po,Dem_candidate,Repub_candidate,Repub%of_two-party_vote,state
1,2016,AL,RON CRUMPTON,RICHARD C. SHELBY,0.640702,ALABAMA
8,2016,AK,RAY METCALFE,LISA MURKOWSKI,0.79237,ALASKA
17,2016,AZ,ANN KIRKPATRICK,JOHN MCCAIN,0.568609,ARIZONA
23,2016,AR,CONNER ELDRIDGE,JOHN BOOZMAN,0.622993,ARKANSAS
26,2016,CA,KAMALA D. HARRIS,LORETTA SANCHEZ,0.383972,CALIFORNIA
...,...,...,...,...,...,...
545,2020,SC,JAIME HARRISON,LINDSEY GRAHAM,0.552079,SOUTH CAROLINA
550,2020,TN,MARQUITA BRADSHAW,BILL HAGERTY,0.638852,TENNESSEE
562,2020,TX,"MARY ""MJ"" HEGAR",JOHN CORNYN,0.549495,TEXAS
567,2020,VA,MARK R. WARNER,DANIEL M. GADE,0.439521,VIRGINIA


<font size = 6>adding CF score equivalents</font>

CF scores are not available for candidates after 2012, so I will use the relationship between DW Nominate and CF Scores to calculate and approximate CF scores for incumbents (since having a DW Nominate score requires having a voting record) and use fast.ai's FillMissing for nonincumbents or names that otherwise can't find an associated score. FillMissing assigns the average value of that column, so a Republican who does not have a DW Nominate score will be treated as an ideologically average Republican. Thus, while there will be many missing values, I expect that will be okay.

In [9]:
# what 'Mapping the Ideological Marketplace' says about CF-DW Nominate relationship: 
# it looks like, from the graphs, cf score ~ dw nom * 2

In [10]:
# building dataset of DW nom scores
dws = pd.read_csv('..\\data\\candidate_ideology\\S116_members.csv') # for 2020 (2019-2021 congress session)

In [11]:
dws

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,116,President,99912,99,0.0,USA,200,0.0,0.0,"TRUMP, Donald John",...,NaN,0.403,0.162,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,116,Senate,41705,41,0.0,AL,100,NaN,NaN,"JONES, Gordon Douglas (Doug)",...,NaN,-0.088,0.187,-80.32594,0.88065,632.0,33.0,NaN,-0.081,0.234
2,116,Senate,94659,41,0.0,AL,200,NaN,NaN,"SHELBY, Richard C.",...,NaN,0.448,0.575,-46.01121,0.93295,663.0,27.0,NaN,0.437,0.219
3,116,Senate,40300,81,0.0,AK,200,NaN,NaN,"MURKOWSKI, Lisa",...,NaN,0.210,-0.302,-47.61779,0.92641,623.0,11.0,NaN,0.283,-0.424
4,116,Senate,41500,81,0.0,AK,200,NaN,NaN,"SULLIVAN, Daniel Scott",...,NaN,0.480,0.063,-53.40271,0.92101,649.0,25.0,NaN,0.510,-0.040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,116,Senate,40915,56,0.0,WV,100,NaN,NaN,"MANCHIN, Joe, III",...,NaN,-0.058,0.420,-70.59255,0.89783,655.0,19.0,NaN,-0.052,0.388
99,116,Senate,29940,25,0.0,WI,100,NaN,NaN,"BALDWIN, Tammy",...,NaN,-0.493,-0.136,-132.31357,0.81982,666.0,65.0,NaN,-0.396,-0.001
100,116,Senate,41111,25,0.0,WI,200,NaN,NaN,"JOHNSON, Ron",...,NaN,0.617,-0.182,-46.18103,0.93070,643.0,24.0,NaN,0.610,0.017
101,116,Senate,40707,68,0.0,WY,200,NaN,NaN,"BARRASSO, John A.",...,NaN,0.536,0.210,-31.02986,0.95461,668.0,15.0,NaN,0.602,0.323


In [13]:
dws = dws.loc[:,['congress','bioname','nominate_dim1']] 

In [14]:
dws2018 = pd.read_csv('..\\data\\candidate_ideology\\S115_members.csv')
dws2018 = dws2018.loc[:,['congress','bioname','nominate_dim1']]
dws2018

,congress,bioname,nominate_dim1
0,115,"TRUMP, Donald John",0.403
1,115,"STRANGE, Luther Johnson, III",0.560
2,115,"JONES, Gordon Douglas (Doug)",-0.088
3,115,"SESSIONS, Jefferson Beauregard III (Jeff)",0.549
4,115,"SHELBY, Richard C.",0.448
...,...,...,...
101,115,"MANCHIN, Joe, III",-0.058
102,115,"BALDWIN, Tammy",-0.493
103,115,"JOHNSON, Ron",0.617
104,115,"BARRASSO, John A.",0.536


In [15]:
dws2016 = pd.read_csv('..\\data\\candidate_ideology\\S114_members.csv')
dws2016 = dws2016.loc[:,['congress','bioname','nominate_dim1']]
dws2016

,congress,bioname,nominate_dim1
0,114,"OBAMA, Barack",-0.358
1,114,"SESSIONS, Jefferson Beauregard III (Jeff)",0.549
2,114,"SHELBY, Richard C.",0.448
3,114,"MURKOWSKI, Lisa",0.210
4,114,"SULLIVAN, Daniel Scott",0.480
...,...,...,...
96,114,"MANCHIN, Joe, III",-0.058
97,114,"BALDWIN, Tammy",-0.493
98,114,"JOHNSON, Ron",0.617
99,114,"BARRASSO, John A.",0.536


In [16]:
dfs = [dws2016,dws2018,dws]
ideologies = pd.concat(dfs)
ideologies

,congress,bioname,nominate_dim1
0,114,"OBAMA, Barack",-0.358
1,114,"SESSIONS, Jefferson Beauregard III (Jeff)",0.549
2,114,"SHELBY, Richard C.",0.448
3,114,"MURKOWSKI, Lisa",0.210
4,114,"SULLIVAN, Daniel Scott",0.480
...,...,...,...
98,116,"MANCHIN, Joe, III",-0.058
99,116,"BALDWIN, Tammy",-0.493
100,116,"JOHNSON, Ron",0.617
101,116,"BARRASSO, John A.",0.536


In [17]:
d = {114:2016, 115:2018, 116: 2020} 
ideologies['congress'].map(d)
ideologies

,congress,bioname,nominate_dim1
0,114,"OBAMA, Barack",-0.358
1,114,"SESSIONS, Jefferson Beauregard III (Jeff)",0.549
2,114,"SHELBY, Richard C.",0.448
3,114,"MURKOWSKI, Lisa",0.210
4,114,"SULLIVAN, Daniel Scott",0.480
...,...,...,...
98,116,"MANCHIN, Joe, III",-0.058
99,116,"BALDWIN, Tammy",-0.493
100,116,"JOHNSON, Ron",0.617
101,116,"BARRASSO, John A.",0.536


In [18]:
ideologies.insert(0, 'year', ideologies['congress'].map(d))
ideologies.drop('congress',axis=1, inplace=True)

In [19]:
ideologies

,year,bioname,nominate_dim1
0,2016,"OBAMA, Barack",-0.358
1,2016,"SESSIONS, Jefferson Beauregard III (Jeff)",0.549
2,2016,"SHELBY, Richard C.",0.448
3,2016,"MURKOWSKI, Lisa",0.210
4,2016,"SULLIVAN, Daniel Scott",0.480
...,...,...,...
98,2020,"MANCHIN, Joe, III",-0.058
99,2020,"BALDWIN, Tammy",-0.493
100,2020,"JOHNSON, Ron",0.617
101,2020,"BARRASSO, John A.",0.536


In [20]:
def dw_to_cf(x): 
    return 2*x

ideologies['approx_cf_score'] = ideologies['nominate_dim1'].map(dw_to_cf)

In [21]:
ideologies.drop('nominate_dim1', axis=1, inplace=True)

In [22]:
elections

,year,state_po,Dem_candidate,Repub_candidate,Repub%of_two-party_vote,state
1,2016,AL,RON CRUMPTON,RICHARD C. SHELBY,0.640702,ALABAMA
8,2016,AK,RAY METCALFE,LISA MURKOWSKI,0.79237,ALASKA
17,2016,AZ,ANN KIRKPATRICK,JOHN MCCAIN,0.568609,ARIZONA
23,2016,AR,CONNER ELDRIDGE,JOHN BOOZMAN,0.622993,ARKANSAS
26,2016,CA,KAMALA D. HARRIS,LORETTA SANCHEZ,0.383972,CALIFORNIA
...,...,...,...,...,...,...
545,2020,SC,JAIME HARRISON,LINDSEY GRAHAM,0.552079,SOUTH CAROLINA
550,2020,TN,MARQUITA BRADSHAW,BILL HAGERTY,0.638852,TENNESSEE
562,2020,TX,"MARY ""MJ"" HEGAR",JOHN CORNYN,0.549495,TEXAS
567,2020,VA,MARK R. WARNER,DANIEL M. GADE,0.439521,VIRGINIA


In [23]:
def extract_last_name(name: str) -> str:
    splt = name.split(' ')
    if splt[-1] in ['III','IV','JR','JR.','SR','SR.']:
        final = splt[-2].replace(',', '') # in case comma after last name before JR./III
        return final
    else:
        return splt[-1]

In [26]:
def find_approxcfscore(name,year):
    last_name = extract_last_name(name)
    matches = ideologies[(ideologies.bioname.str.contains(last_name,regex=True))]
    if len(matches)>=1:
        return matches.iloc[0,2]
    else:
        return None

In [27]:
ideologies['bioname'] = ideologies['bioname'].str.upper()

In [28]:
elections['Dem_cfscore'] = elections.apply(lambda x: find_approxcfscore(x.Dem_candidate, x.year), axis=1)
elections['Repub_cfscore'] = elections.apply(lambda x: find_approxcfscore(x.Repub_candidate,x.year), axis=1)

<font size = 6> adding PVIs</font>

In [29]:
pvis2016 = pd.read_csv('..\\data\\state_partisan_lean\\Cook Political Report PVI Scores, Before and After 2012 Election - States.csv')

In [30]:
pvis2016

,"Sources: Polidata, Cook Political Report, U.S. Census Bureau",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,State,Electoral Votes,2012 PVI,2016 PVI,Dem Shift
2,Alabama,9,R+13,R+14,-0.76
3,Alaska,3,R+13,R+12,1.35
4,Arizona,11,R+6,R+7,-1.27
5,Arkansas,6,R+9,R+14,-5.21
6,California,55,D+7,D+9,1.81
7,Colorado,9,EVEN,D+1,0.95
8,Connecticut,7,D+7,D+7,0.15
9,Delaware,3,D+7,D+8,1.2


In [31]:
pvis2016.drop(columns={'Unnamed: 1','Unnamed: 2','Unnamed: 4'}, inplace=True)

In [32]:
pvis2016.columns=pvis2016.iloc[1]
pvis2016=pvis2016.iloc[2:]
pvis2016

1,State,2016 PVI
2,Alabama,R+14
3,Alaska,R+12
4,Arizona,R+7
5,Arkansas,R+14
6,California,D+9
7,Colorado,D+1
8,Connecticut,D+7
9,Delaware,D+8
10,District of Columbia,D+40
11,Florida,R+2


In [35]:
def signed_pvi(x):
    s = str(x)
    splt = s.split('+')
    if splt[0]=='R':
        return splt[1]
    elif splt[0]=='EVEN':
        return 0
    else:
        return -1*int(splt[1])

In [36]:
pvis2016['signed_pvi'] = pvis2016['2016 PVI'].apply(signed_pvi)

C:\Users\bbdil\AppData\Local\Temp\ipykernel_27864\1339752428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pvis2016['signed_pvi'] = pvis2016['2016 PVI'].apply(signed_pvi)


In [37]:
pvis2016['State'] = pvis2016['State'].str.upper()
pvis2016

C:\Users\bbdil\AppData\Local\Temp\ipykernel_27864\224562334.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pvis2016['State'] = pvis2016['State'].str.upper()


1,State,2016 PVI,signed_pvi
2,ALABAMA,R+14,14
3,ALASKA,R+12,12
4,ARIZONA,R+7,7
5,ARKANSAS,R+14,14
6,CALIFORNIA,D+9,-9
7,COLORADO,D+1,-1
8,CONNECTICUT,D+7,-7
9,DELAWARE,D+8,-8
10,DISTRICT OF COLUMBIA,D+40,-40
11,FLORIDA,R+2,2


In [47]:
pvis2016.index = pvis2016.State

In [48]:
pvis2016

1,State,2016 PVI,signed_pvi
State,,,
ALABAMA,ALABAMA,R+14,14
ALASKA,ALASKA,R+12,12
ARIZONA,ARIZONA,R+7,7
ARKANSAS,ARKANSAS,R+14,14
CALIFORNIA,CALIFORNIA,D+9,-9
COLORADO,COLORADO,D+1,-1
CONNECTICUT,CONNECTICUT,D+7,-7
DELAWARE,DELAWARE,D+8,-8
DISTRICT OF COLUMBIA,DISTRICT OF COLUMBIA,D+40,-40


In [43]:
# don't need 2018 because in training set i did pvi every 4 years
pvis2020 = pd.read_csv('..\\data\\state_partisan_lean\\fivethirtyeight_partisan_lean_2020STATES.csv') 

In [44]:
pvis2020

,state,2020
0,Alabama,R+27
1,Alaska,R+16
2,Arizona,R+8
3,Arkansas,R+26
4,California,D+22
5,Colorado,R+0
6,Connecticut,D+10
7,Delaware,D+12
8,Florida,R+6
9,Georgia,R+13


In [45]:
pvis2020['signed pvi'] = pvis2020['2020'].apply(signed_pvi)
pvis2020['state'] = pvis2020['state'].str.upper()

In [49]:
pvis2020.index = pvis2020.state

In [50]:
pvis2020

,state,2020,signed pvi
state,,,
ALABAMA,ALABAMA,R+27,27
ALASKA,ALASKA,R+16,16
ARIZONA,ARIZONA,R+8,8
ARKANSAS,ARKANSAS,R+26,26
CALIFORNIA,CALIFORNIA,D+22,-22
COLORADO,COLORADO,R+0,0
CONNECTICUT,CONNECTICUT,D+10,-10
DELAWARE,DELAWARE,D+12,-12
FLORIDA,FLORIDA,R+6,6


Here is where information from these datasets gets added to my Test Set

In [51]:
def get_pvi(state,year):
    if year%4!=0: # pvis are only calculated once every 4 years, but i have midterm elections in my data set
        year-=2 # use pvi for most recent presidential election year
    if year==2016:
        return pvis2016.loc[state,'signed_pvi']
    elif year==2020:
        return pvis2020.loc[state, 'signed pvi'] # notice one has an underscore and one doesn't

In [52]:
elections['state enviro'] = elections.apply(lambda x: get_pvi(x.state,x.year), axis=1)

In [53]:
elections

,year,state_po,Dem_candidate,Repub_candidate,Repub%of_two-party_vote,state,Dem_cfscore,Repub_cfscore,state enviro
1,2016,AL,RON CRUMPTON,RICHARD C. SHELBY,0.640702,ALABAMA,NaN,0.896,14
8,2016,AK,RAY METCALFE,LISA MURKOWSKI,0.79237,ALASKA,NaN,0.420,12
17,2016,AZ,ANN KIRKPATRICK,JOHN MCCAIN,0.568609,ARIZONA,NaN,0.762,7
23,2016,AR,CONNER ELDRIDGE,JOHN BOOZMAN,0.622993,ARKANSAS,NaN,0.852,14
26,2016,CA,KAMALA D. HARRIS,LORETTA SANCHEZ,0.383972,CALIFORNIA,-1.418,NaN,-9
...,...,...,...,...,...,...,...,...,...
545,2020,SC,JAIME HARRISON,LINDSEY GRAHAM,0.552079,SOUTH CAROLINA,NaN,0.758,16
550,2020,TN,MARQUITA BRADSHAW,BILL HAGERTY,0.638852,TENNESSEE,NaN,NaN,28
562,2020,TX,"MARY ""MJ"" HEGAR",JOHN CORNYN,0.549495,TEXAS,NaN,0.948,17
567,2020,VA,MARK R. WARNER,DANIEL M. GADE,0.439521,VIRGINIA,-0.408,NaN,-7


<font size = 6> Adding natl enviro and economy</font>

In [55]:
# set this up to allow for adding each state's unemployment later
info_dict = {
    2016: {'natl enviro': 1.1, # this is actual house pop vote margin, not polls. From wikipedia
           'inflation': 1.26,
           'natl unemploy': 4.87,
          },
    2018: {'natl enviro': -8.6, 
           'inflation': 2.44,
           'natl unemploy': 3.90,
          },
    2020: {'natl enviro': -3.1,
           'inflation': 1.23,
           'natl unemploy': 8.05,
          }
}

In [58]:
elections['Natl_enviro'] = elections.apply(lambda x: info_dict[x.year]['natl enviro'], axis=1)
elections['Inflation'] = elections.apply(lambda x: info_dict[x.year]['inflation'], axis=1)
elections['Natl_unempl'] = elections.apply(lambda x: info_dict[x.year]['natl unemploy'], axis=1)

In [59]:
elections

,year,state_po,Dem_candidate,Repub_candidate,Repub%of_two-party_vote,state,Dem_cfscore,Repub_cfscore,state enviro,Natl_enviro,Inflation,Natl_unempl
1,2016,AL,RON CRUMPTON,RICHARD C. SHELBY,0.640702,ALABAMA,NaN,0.896,14,1.1,1.26,4.87
8,2016,AK,RAY METCALFE,LISA MURKOWSKI,0.79237,ALASKA,NaN,0.420,12,1.1,1.26,4.87
17,2016,AZ,ANN KIRKPATRICK,JOHN MCCAIN,0.568609,ARIZONA,NaN,0.762,7,1.1,1.26,4.87
23,2016,AR,CONNER ELDRIDGE,JOHN BOOZMAN,0.622993,ARKANSAS,NaN,0.852,14,1.1,1.26,4.87
26,2016,CA,KAMALA D. HARRIS,LORETTA SANCHEZ,0.383972,CALIFORNIA,-1.418,NaN,-9,1.1,1.26,4.87
...,...,...,...,...,...,...,...,...,...,...,...,...
545,2020,SC,JAIME HARRISON,LINDSEY GRAHAM,0.552079,SOUTH CAROLINA,NaN,0.758,16,-3.1,1.23,8.05
550,2020,TN,MARQUITA BRADSHAW,BILL HAGERTY,0.638852,TENNESSEE,NaN,NaN,28,-3.1,1.23,8.05
562,2020,TX,"MARY ""MJ"" HEGAR",JOHN CORNYN,0.549495,TEXAS,NaN,0.948,17,-3.1,1.23,8.05
567,2020,VA,MARK R. WARNER,DANIEL M. GADE,0.439521,VIRGINIA,-0.408,NaN,-7,-3.1,1.23,8.05


In [47]:
# Saving the dataset to csv so can load in other notebooks
elections.to_csv('..\\prepared_data\\test_set_v1.csv',index=False)